In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def grow_axon(phi0, n_rho=801, rho_range=(4.0, 45.0), loc_od=(15.0, 2.0),
              beta_sup=-1.9, beta_inf=0.5):
    """Grows a single axon based on the model by Jansonius et al. (2009)

    This function generates the trajectory of a single nerve fiber bundle based
    on the mathematical model described in:

    > Jansionus et al. (2009). A mathematical description of nerve fiber bundle
    > trajectories and their variability in the human retina. Vis Res 49:
    > 2157-2163.

    Parameters
    ----------
    phi0 : float
        Angular position of the axon at its starting point (polar coordinates,
        degrees). Must be within [-180, 180].
    n_rho : int, optional, default: 801
        Number of sampling point along the radial axis (polar coordinates).
    rho_range : (rho_min, rho_max), optional, default: (4.0, 45.0)
        Lower and upper bounds for the radial position values (polar
        coordinates).
    loc_od : (x_od, y_od), optional, default: (15.0, 2.0)
        Location of the center of the optic disc (x, y) in Cartesian
        coordinates.
    beta_sup : float, optional, default: -1.9
        Scalar value for the superior retina (see Eq. 5, `\beta_s` in the
        paper).
    beta_inf : float, optional, default: 0.5
        Scalar value for the inferior retina (see Eq. 6, `\beta_i` in the
        paper.)
    """
    if np.abs(phi0) > 180.0:
        raise ValueError('phi0 must be within [-180, 180].')
    if n_rho < 1:
        raise ValueError('Number of radial sampling points must be >= 1.')
    if np.any(np.abs(rho_range) < 0):
        raise ValueError('rho cannot be negative.')
    if rho_range[0] > rho_range[1]:
        raise ValueError('Lower bound on rho cannot be larger than the '
                         ' upper bound.')
    is_superior = phi0 > 0
    if is_superior:
        # Axon is in superior retina, compute `b` (real number) from Eq. 5:
        b = np.exp(beta_sup + 3.9 * np.tanh(-(phi0 - 121.0) / 14.0))
        # Equation 3, `c` a positive real number:
        c = 1.9 + 1.4 * np.tanh((phi0 - 121.0) / 14.0)
    else:
        # Axon is in inferior retina: compute `b` (real number) from Eq. 6:
        b = -np.exp(beta_inf + 1.5 * np.tanh(-(-phi0 - 90.0) / 25.0))
        # Equation 4, `c` a positive real number:
        c = 1.0 + 0.5 * np.tanh((-phi0 - 90.0) / 25.0)

    # Spiral as a function of `rho`:
    phi = phi0 + b * (rho - rho.min()) ** c

    # Convert to Cartesian coordinates
    xprime = rho * np.cos(np.deg2rad(phi))
    yprime = rho * np.sin(np.deg2rad(phi))

    # Find the array elements where the axon crosses the meridian
    if is_superior:
        # Find elements in inferior retina
        idx = np.where(yprime < 0)[0]
    else:
        # Find elements in superior retina
        idx = np.where(yprime > 0)[0]
    if idx.size:
        # Keep only up to first occurrence
        xprime = xprime[:idx[0]]
        yprime = yprime[:idx[0]]

    # From `loc_od`=[0, 0] to fovea=[0, 0]
    xmodel = xprime + loc_od[0]
    ymodel = yprime
    idx = xprime > -loc_od[0]
    ymodel[idx] = yprime[idx] + loc_od[1] * (xmodel[idx] / loc_od[0]) ** 2

    return xmodel, ymodel

In [4]:
import numpy as np
for sign in [-1.0, 1.0]:
    for phi0 in [110.0, 135.0, 160.0]:
        x, y = grow_axon(sign * phi0, n_rho=500, rho_range=(0.0, 45.0))
        np.isclose(y[-1], 0.0)

NameError: name 'rho' is not defined

In [ ]:
for phi0 in np.linspace(-180, 180, 100):
    x, y = grow_axon(phi0, n_rho=500, rho_range=(0.0, 45.0))
    plt.plot(x, y)

In [ ]:
from pulse2percept.utils import parfor
axons = parfor(grow_axon, np.linspace(-180, 180, 100), engine='joblib')